# LangChain Expression Language (LCEL) basics

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

Uncomment & run the code in the cell below:

In [ ]:
# pip install langchain

## 1. The '|' operator

* Equivalent to the __or__ operator
* In Python, this operator can be overloaded
* LangChain framework has overloaded it to achieve a UNIX pipe like behavior!!

In [1]:
# https://docs.python.org/3/library/operator.html#operator.__or__
from operator import __or__

number_1 = 1
number_2 = 5

bool_value = __or__(number_1, number_2) > 4

bool_value

True

#### Expression __ __or__ __(number_1, number_2) is equivalent to   (number_1  |  number_2)

In [2]:
bool_value = number_1 | number_2 > 4

bool_value

True

#### Python operators can be overloaded

## 2. LangChain has overridden the '|'   __ __or__ __ operator

* All components of the chain implement the **invoke(..)** methods
* The **input/output types** for the **invoke function** varies depending on the type of the component
* Component exposes the *input/output schema**
* User/application input is fed as input for the first component
* Output of each component in the chain is fed as input into the next component in the chain
* Output from the last component is the final output from the chain

#### Runnables
https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.runnables

https://python.langchain.com/v0.1/docs/expression_language/interface/



#### RunnableLambda

Convert a Python function into a runnable.

https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html#langchain_core.runnables.base.RunnableLambda

In [1]:
from langchain_core.runnables import RunnableLambda

# Example function to be wrapped in runnable
def add_one(x: int) -> int:
    return x + 1

# Create the runnable
add_one_runnable = RunnableLambda(func = add_one)

# Print the name
print("Name: ", add_one_runnable.name)

# Invoke and print result
print("Invoke with 10: ", add_one_runnable.invoke(10))

Name:  add_one
Invoke with 10:  11


#### RunnablePassthrough

https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html#langchain_core.runnables.passthrough.RunnablePassthrough

* Takes no action on the input data
* May add keys to the input dictionary
* Used in combination with other *Runnables*

In [4]:
from langchain_core.runnables import RunnablePassthrough

# Create a pass through runnable
do_nothing_runnable = RunnablePassthrough()

# Do nothing runnable
input = {"a": 1, "b": 2}
print("Do nothing : ", do_nothing_runnable.invoke(input))

# Add new key 'c' with a fixed value
add_new_keys_runnable = RunnablePassthrough.assign(c=lambda inputs: 3)
print("Add keys : ", add_new_keys_runnable.invoke(input))


Do nothing :  {'a': 1, 'b': 2}
Add keys :  {'a': 1, 'b': 2, 'c': 3}


#### RunnableSequence

https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html#langchain_core.runnables.base.RunnableSequence

In [5]:
from langchain_core.runnables import RunnableSequence

# Create 3 example functions & corrsponding RunnableLambda
def add_1(x: int) -> int:
    return x + 1

add_1_runnable = RunnableLambda(func = add_1)

def multiply_2(x: int) -> int:
    return x*2

multiply_2_runnable = RunnableLambda(func = multiply_2)

def subtract_3(x: int) -> int:
    return x - 1

subtract_3_runnable = RunnableLambda(func = subtract_3)

In [6]:
# Two ways to create a runnable sequence

# 1. Using the pipe/or operator. This is used more commonly
runnable_sequence = add_1_runnable | multiply_2_runnable | subtract_3_runnable

# 2. Directly instantiate
runnable_sequence = RunnableSequence(add_1_runnable, multiply_2_runnable, subtract_3_runnable)

# Invoke with a input = 1
# x=1  (1+1)*2 - 1 = 3
print("Input = 1, Output = ", runnable_sequence.invoke(1))

# Invoke with a input = 5
# x=5  (5+1)*2 - 1 = 11
print("Input = 5, Output = ", runnable_sequence.invoke(5))



Input = 1, Output =  3
Input = 5, Output =  11


#### RunnableParallel

https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html#langchain_core.runnables.base.RunnableParallel

In [7]:
%%time

from langchain_core.runnables import RunnableParallel


# Adding a pasthrough to have the original input deleievered in the output
runnable_parallel = RunnableParallel(   
                                        original=RunnablePassthrough(),
                                        add_1=add_1_runnable,
                                        multiply_2=multiply_2_runnable,
                                        subtract_3=subtract_3_runnable
                                    )

runnable_parallel.invoke(5)

CPU times: total: 0 ns
Wall time: 17 ms


{'original': 5, 'add_1': 6, 'multiply_2': 10, 'subtract_3': 4}

##### RunnableParallel doesn't return till all runnables have completed execution

In [8]:
%%time

import time

# This function simulates slow processing
def   simulate_slow_processing_for_add_1(x: int) -> int:
    for i in range(10):
        time.sleep(1)
    return x+1

# Runnable for slow lambda
add_1_slow_processing = RunnableLambda(simulate_slow_processing_for_add_1)

runnable_parallel = RunnableParallel(   
                                        original=RunnablePassthrough(),
                                        add_1=add_1_slow_processing,
                                        multiply_2=multiply_2_runnable,
                                        subtract_2=subtract_3_runnable
                                    )
runnable_parallel.invoke(5)

CPU times: total: 0 ns
Wall time: 10 s


{'original': 5, 'add_1': 6, 'multiply_2': 10, 'subtract_2': 4}

#### RunnableBranch

https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.branch.RunnableBranch.html#langchain_core.runnables.branch.RunnableBranch

Following code implements the scenario:
1. Input has an {action, number}
2. A function (add_1, multiply_2, subtract_3) is applied based on action
3. A default function is run if the action key is not provided or is invalid

In [44]:
from langchain_core.runnables import RunnableBranch

# action key not found
def error_condition(x: dict) -> dict:
    if "action" not in x:
        x["error"] = "Action NOT provided"
    else:
        x["error"] = "Invalid action. Only 'add', 'multiply' & 'subtract are supported"
        
    return x

# Create a default runnable
default_runnable = RunnableLambda(error_condition)

# Takes dictionary as input and returns dict as output
def add_1_dict(x: dict) -> dict:
    x["output"] = x["number"] + 1
    return x

def multiply_2_dict(x: dict) -> dict:
    x["output"] = x["number"]*2
    return x

def subtract_3_dict(x: dict) -> dict:
    x["output"] = x["number"] - 1
    return x

# Define the branches as (condition, runnable) tuples
branch = RunnableBranch(
    (lambda input: input.get("action", "NA") == "add", RunnableLambda(add_1_dict)),
    (lambda input: input.get("action", "NA") == "multiply", RunnableLambda(multiply_2_dict)),
    (lambda input: input.get("action", "NA") == "subtract", RunnableLambda(subtract_3_dict)),
    default_runnable
)

# Change the action (add, multiply, subtract, invalid) to see the branching
response = branch.invoke({ "action": "invalid", "number": 5}) 

response

{'action': 'invalid',
 'number': 5,
 'error': "Invalid action. Only 'add', 'multiply' & 'subtract are supported"}